- .[OLS](#ols)
- .[一些统计量的计算方法](#stats)

# OLS的一些统计量<a id='stats'></a>

In [3]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm

In [8]:
import random

n_sample = 20
age = [random.randint(12, 18) for _ in range(n_sample)]
gender = [random.randint(0, 1) for _ in range(n_sample)]
height = [165 - g + (a - 12) * (g + 1) + random.random() for a, g in zip(age, gender)]
df = pd.DataFrame({
    'age': age,
    'gender': gender,
    'height': height
})
df

,age,gender,height
0,13,1,166.746936
1,15,0,168.631229
2,18,1,176.332543
3,16,1,172.109904
4,12,0,165.381427
5,16,1,172.416753
6,17,0,170.653608
7,15,0,168.579097
8,18,0,171.587899
9,16,1,172.597424


In [11]:
df['constant'] = 1
X = df[['age', 'gender', 'constant']]
y = df['height']
model = sm.OLS(endog=y, exog=X)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 height   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     100.3
Date:                Sun, 26 Nov 2023   Prob (F-statistic):           3.87e-10
Time:                        19:10:24   Log-Likelihood:                -25.544
No. Observations:                  20   AIC:                             57.09
Df Residuals:                      17   BIC:                             60.07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age            1.2951      0.110     11.748      0.000       1.062       1.528
gender         2.3678      0.431      5.490      0.000       1.458       3.278
constant     149.1755      1.668     89.439      0.000     145.657     152.694
==============================================================================
Omnibus:                        0.305   Durbin-Watson:                   1.349
Prob(Omnibus):                  0.858   Jarque-Bera (JB):                0.468
Skew:                          -0.028   Prob(JB):                        0.791
Kurtosis:                       2.252   Cond. No.                         122.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
result.pvalues

age         1.391776e-09
gender      3.980035e-05
constant    3.595577e-24
dtype: float64

$$ 
y = \beta * X + \epsilon \\
\hat{\beta}^{OLS}=\underset{\beta}{\operatorname{argmin}} \| \epsilon \|^2 \\
\hat{\beta} = (X^t * X)^{-1} * X^t * y 
$$

In [15]:
xt = np.transpose(X)
xt_x = xt.dot(X)
inverse = np.linalg.inv(xt_x)
beta = inverse.dot(xt).dot(y)
beta

array([  1.29506783,   2.36776739, 149.17550539])

$$ Var( \hat{\beta})  \\
= Var((X^t * X)^{-1} * X^t * y)  \\
= (X^t * X)^{-1} * X^t* Var(y) * ((X^t * X)^{-1} * X^t)^t \\
= (X^t * X)^{-1} * X^t* \sigma^2 * ((X^t * X)^{-1} * X^t)^t \\
= \sigma^2 *(X^t * X)^{-1} * X^t* X * ((X^t * X)^{-1})^t \\
= \sigma^2 * ((X^t * X)^{-1})^t \\ 
= \sigma^2 * (X * X^t)^{-1}
$$

In [19]:
residual = y - X.dot(beta)
dof = len(y) - X.shape[1]
var = sum(residual ** 2) / df
var_beta = var * np.transpose(inverse)
var, var_beta

(0.8860538078358016,
 array([[ 0.01215286, -0.00920671, -0.18118806],
        [-0.00920671,  0.18597555,  0.05671334],
        [-0.18118806,  0.05671334,  2.78189961]]))

In [18]:
std = np.diag(var_beta) ** 0.5
std

array([0.11024   , 0.43124883, 1.66790276])

In [20]:
from scipy import stats
stats.t.sf(beta / std, dof) * 2

array([1.39177570e-09, 3.98003511e-05, 3.59557718e-24])